# 2.4
Separate notebook as `ch2.ipynb` got large and unwieldly/slow.

## Complex-Number System

The complex number $z=x+iy$ (where $i^2=-1$) can be thought of as the point in the plane whose *real* coordinate is $x$ and whose *imaginary* coordinate is $y$.

Can have 2 representations:
- Reactangular form
    - Real part and imaginary part
- Polar form
    - Magnitude and angle
    
![](images/complex-numbers.png)

#### Addition 
Of two complex numbers $z_1,z_2$ reduces to addition of coordinates - using rectangular form:

$RealPart(z_1+z_2)=RealPart(z_1)+RealPart(z_2)$

$ImaginaryPart(z_1+z_2)=ImaginaryPart(z_1)+ImaginaryPart(z_2)$

#### Product 
Of two complex numbers $z_1,z_2$ is the vector obtained by stretching one complex number by the length of the other and then rotating it throught the angle of the other - using polar form:

$Magnitude(z_1\cdot z_2) = Magnitude(z_1)\cdot Magnitude(z_2)$

$Angle(z_1\cdot z_2) = Angle(z_1)+Magnitude(z_2)$

In [1]:
;; asumming that put and get procedures are implemented in the language
(define (install-rectangular-package)  
  ;; internal procedures
  (define (real-part z) (car z))
  (define (imag-part z) (cdr z))
  (define (make-from-real-imag x y)
    (cons x y))
  (define (magnitude z)
    (sqrt (+ (square (real-part z))
             (square (imag-part z)))))
  (define (angle z)
    (atan (imag-part z) (real-part z)))
  (define (make-from-mag-ang r a)
    (cons (* r (cos a)) (* r (sin a))))
  ;; interface to the rest of the system
  (define (tag x)
    (attach-tag 'rectangular x))
  (put 'real-part '(rectangular) real-part)
  (put 'imag-part '(rectangular) imag-part)
  (put 'magnitude '(rectangular) magnitude)
  (put 'angle '(rectangular) angle)
  (put 'make-from-real-imag 'rectangular
       (lambda (x y)
         (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'rectangular
       (lambda (r a)
         (tag (make-from-mag-ang r a))))
  'done)

(define (install-polar-package)
  ;; internal procedures
  (define (magnitude z) (car z))
  (define (angle z) (cdr z))
  (define (make-from-mag-ang r a) (cons r a))
  (define (real-part z)
    (* (magnitude z) (cos (angle z))))
  (define (imag-part z)
    (* (magnitude z) (sin (angle z))))
  (define (make-from-real-imag x y)
    (cons (sqrt (+ (square x) (square y)))
          (atan y x)))
  ;; interface to the rest of the system
  (define (tag x) (attach-tag 'polar x))
  (put 'real-part '(polar) real-part)
  (put 'imag-part '(polar) imag-part)
  (put 'magnitude '(polar) magnitude)
  (put 'angle '(polar) angle)
  (put 'make-from-real-imag 'polar
       (lambda (x y)
         (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'polar
       (lambda (r a)
         (tag (make-from-mag-ang r a))))
  'done)